#### Sample API query
http://localhost:8080/otp/routers/default/plan?fromPlace=57.68845%2C11.93338&toPlace=57.67121%2C11.93156&time=1%3A02pm&date=11-10-2022&mode=TRANSIT%2CWALK&maxWalkDistance=5000&arriveBy=false


In [1]:
import pandas as pd
import requests 
from mainCopy import make_query, parse_OTP_response
from utils import plot
import numpy as np

## 1. Querying

In [10]:
PATH = 'Q/georequestsSummer.csv'
OTP_API = "http://localhost:8080/otp/routers/default/plan"

In [15]:
df = pd.read_csv(PATH, index_col=[0], skiprows=[i for i in range(1,70001)])# skipping first n+1 (since n does not include the given row) rows since they were successful in the previous run
#df = pd.read_csv(PATH)  # load the csv #index_col=[0]
df.treq=pd.to_datetime(df.treq)

print("Total trips in dataframe:", len(df))
df

Total trips in dataframe: 172277


,origin_x,origin_y,destination_x,destination_y,treq,END_TIME,duration_in_m
TRIP_ID,,,,,,,
542322,12.001319,57.714793,12.008305,57.714067,2022-06-29 21:54:46,2022-06-22 21:57:23,2.616667
542323,11.985716,57.700373,11.971456,57.712969,2022-06-29 21:55:01,2022-06-22 22:02:13,7.200000
542324,11.986538,57.700052,11.973469,57.707337,2022-06-29 21:55:08,2022-06-22 22:02:51,7.716667
542325,11.934732,57.698272,11.937314,57.691658,2022-06-29 21:55:39,2022-06-22 21:59:17,3.633333
542326,11.959720,57.696393,11.974631,57.699691,2022-06-29 21:56:20,2022-06-22 22:01:03,4.716667
...,...,...,...,...,...,...,...
751508,11.935584,57.669435,11.955417,57.693824,2022-07-03 23:56:07,2022-08-15 00:06:12,10.083333
751509,11.948698,57.699330,11.977734,57.698754,2022-07-03 23:58:51,2022-08-15 00:12:05,13.233333
751510,11.921802,57.719043,11.932929,57.719492,2022-07-03 23:59:12,2022-08-15 00:02:01,2.816667


### Querying API
*Ensure Grizzly server is running*

java -Xmx4G -jar otp.jar --load inputFiles

In [17]:
row = df.head(1).squeeze()
query = make_query(row)
r = requests.get(OTP_API, params=query) #route r as response from API
route = parse_OTP_response(r.json()) #route r parsed into JSON
route

{'success': True,
 'treq_r': 1656532440000,
 'n_itineraries': 1,
 'startTime': 1656532440000,
 'endTime': 1656532902000,
 'duration': 462,
 'walkDistance': 541.5070000000001,
 'transfers': 0,
 'transitTime': 0,
 'waitingTime': 0,
 'modes': [['WALK', 1656532440000, 1656532902000, 462, 541]]}

In [14]:
routeList = []

for index, row in df.iterrows():
    query = make_query(row)
    r = requests.get(OTP_API, params=query) #route r as response from API
    route = parse_OTP_response(r.json()) #route r parsed into JSON
    routList=routeList.append(route)
    
dfResp = pd.DataFrame(routeList)
dfResp.to_csv('R/responses.csv')


In [26]:
dfResp['success'].value_counts()[False]
#dfResp

1916